Importando bibliotecas


In [ ]:
import numpy as np
import cv2

Importando a imagem de teste

In [ ]:
target = 'img.jpeg'
input_image = cv2.imread(target) 

scale_percentual = 20
width = int(input_image.shape[1] * scale_percentual/100)
height = int(input_image.shape[0] * scale_percentual/100)
dimension = (width, height)

cv2.imshow('Original Image', cv2.resize(input_image, dimension, interpolation = cv2.INTER_AREA))
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('processed/input_image.jpg', input_image)

Verificando as cores dominantes na imagem inicial

In [ ]:
def create_bar(height, width, color):
    bar = np.zeros((height, width, 3), np.uint8)
    bar[:] = color
    red, green, blue = int(color[2]), int(color[1]), int(color[0])
    return bar, (red, green, blue)

In [ ]:
input_image_height, input_image_width, _ = np.shape(input_image)
input_image_data = np.reshape(input_image, (input_image_height * input_image_width, 3))
input_image_data = np.float32(input_image_data)

Clusterização das cores dominantes

https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html

In [ ]:
clusters = 5

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS
input_image_compactness, input_image_labels, input_image_centers = cv2.kmeans(input_image_data, clusters, None, criteria, 10, flags)
print(input_image_centers)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
bars = []
rgb_values = []

for index, color in enumerate(input_image_centers):
    bar, rgb = create_bar(200, 200, color)
    bars.append(bar)
    rgb_values.append(rgb)

input_image_bar = np.hstack(bars)

for index, color in enumerate(rgb_values):
    image = cv2.putText(input_image_bar, f'{index + 1}. RGB: {color}', (5 + 200 * index, 200 - 10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)


cv2.imshow('Original Image', cv2.resize(input_image, dimension, interpolation = cv2.INTER_AREA))
cv2.imshow('Dominant Colors', input_image_bar)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('processed/input_image_dominant_colors.jpg', input_image_bar)

Criando a mascaras para obter as sementes individualmente e para calcular a área das sementes

In [ ]:
# Acentuando cores
used_threshold, thresholded_bgr_image = cv2.threshold(input_image, 130, 255, cv2.THRESH_BINARY)
thresholded_blue_component, thresholded_green_component, thresholded_red_component = cv2.split(thresholded_bgr_image)

# cv2.imshow('Original Image', cv2.resize(input_image, dimension, interpolation = cv2.INTER_AREA))
# cv2.imshow('Thresholded BGR Image', cv2.resize(thresholded_bgr_image, dimension, interpolation = cv2.INTER_AREA))
# cv2.imshow('R', cv2.resize(thresholded_red_component, dimension, interpolation = cv2.INTER_AREA))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# cv2.imwrite('processed/thresholded_bgr_image.jpg', thresholded_bgr_image)
# cv2.imwrite('processed/thresholded_red_component.jpg', thresholded_red_component)

In [ ]:
result_image = cv2.bitwise_and(input_image, input_image, mask = thresholded_red_component)
# cv2.imshow('Result Image', cv2.resize(result_image, dimension, interpolation = cv2.INTER_AREA))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imwrite('processed/result_image.jpg', result_image)

In [ ]:
result_image_height, result_image_width, _ = np.shape(result_image)
result_image_data = np.reshape(result_image, (result_image_height * result_image_width, 3))
result_image_data = np.float32(result_image_data)

In [ ]:
clusters = 5

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS
result_image_compactness, result_image_labels, result_image_centers = cv2.kmeans(result_image_data, clusters, None, criteria, 10, flags)
print(result_image_centers)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
bars = []
rgb_values = []

for index, color in enumerate(result_image_centers):
    bar, rgb = create_bar(200, 200, color)
    bars.append(bar)
    rgb_values.append(rgb)

result_image_bar = np.hstack(bars)

for index, color in enumerate(rgb_values):
    image = cv2.putText(result_image_bar, f'{index + 1}. RGB: {color}', (5 + 200 * index, 200 - 10),
    font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

cv2.imshow('Result Image', cv2.resize(result_image, dimension, interpolation = cv2.INTER_AREA))
cv2.imshow('Dominant Colors', result_image_bar)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('processed/result_image_dominant_colors.jpg', result_image_bar)